In [1]:
import pandas as pd
import requests
import time
import datetime
import urllib.parse
import csv
import os
import requests
import Funciones.tratarJsonYCSV as FncJsonCSV
from bs4 import BeautifulSoup
from modules.oracle import OracleDataInserter

In [2]:
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
BERA_CONEXION = "karstec"
BERA_ESQUEMA = "KARSTEC"

bera_con = OracleDataInserter(schema=BERA_ESQUEMA, glue_con=BERA_CONEXION)

In [3]:
listado = bera_con.ejecutar_consulta_sql(
            sql=f"""
                SELECT *
                FROM CUIT_ONLINE_PBUS
                WHERE CUIT != 'No encontrado'
            """)

datos_CO = listado
columna_resp_pago = ('NOMBRE','APELLIDO','CUIT')

df_cuit_online = pd.DataFrame(datos_CO, columns=columna_resp_pago)

In [ ]:
df_cuit_online

In [5]:
import urllib.parse
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

def busqueda_final(df_cuit_online):
    datos_finales = []  # Lista para acumular los datos recolectados
    
    # Dividir el DataFrame en lotes de 10 filas
    num_lotes = len(df_cuit_online) // 10 + (1 if len(df_cuit_online) % 10 > 0 else 0)
    print(f'Se armaron {num_lotes} lotes para la búsqueda final.')
    for i in range(0, len(df_cuit_online), 10):
        lotes = df_cuit_online.iloc[i:i+10]  # Obtener el lote actual
        for index, row in lotes.iterrows():
            nombre = row['NOMBRE']
            apellido = row['APELLIDO']
            cuit = row['CUIT']

            if cuit:  # Solo intenta la búsqueda si el CUIT existe
                apellido_codificado = urllib.parse.quote(apellido)  # Codificar el apellido
                url = f'https://www.cuitonline.com/detalle/{cuit}/{apellido_codificado}-{nombre}.html'
                try:
                    response = requests.get(url)
                    response.raise_for_status()  # Asegura que no haya errores en la respuesta
                except Exception as e:
                    print(f"Error al obtener la respuesta: {e}")
                encoding = response.encoding  
                response_content = response.content.decode(encoding)

                time.sleep(30)

                soup = BeautifulSoup(response_content, 'html.parser')
                persona_data = soup.find('div', class_='persona-data')
                if persona_data:
                    provincia_elem = persona_data.find('span', itemprop='addressRegion')
                    localidad_elem = persona_data.find('span', itemprop='addressLocality')
                    # Verifica si los elementos de provincia y localidad existen antes de acceder a sus atributos
                    provincia = provincia_elem.text.strip() if provincia_elem else "No encontrado"
                    localidad = localidad_elem.text.strip() if localidad_elem else "No encontrado"
                    datos_finales.append({"nombre": nombre, "apellido": apellido, "cuit": cuit, "provincia": provincia, "localidad": localidad})
                else:
                    datos_finales.append({"nombre": nombre, "apellido": apellido, "cuit": cuit, "provincia": "No encontrado", "localidad": "No encontrado"})

        print(f'Se va procesando lote {i//10 + 1} de {num_lotes} en la búsqueda final.')

    return pd.DataFrame(datos_finales)

In [6]:
datos_finales = busqueda_final(df_cuit_online)

Se armaron 2 lotes para la búsqueda final.
Se va procesando lote 1 de 2 en la búsqueda final.
Se va procesando lote 2 de 2 en la búsqueda final.


In [7]:
datos_finales

,nombre,apellido,cuit,provincia,localidad
0,HINOJO,ESTER,27256505792,No encontrado,No encontrado
1,HINOJO,ESTER,23057475404,Salta,Salta
2,FATTORE,JUAN,20228844676,Buenos Aires,Quilmes
3,FATTORE,JUAN,20304223899,Buenos Aires,Vedia
4,FATTORE,JUAN,30623008858,Cordoba,Ballesteros
5,SOLIS,EDUARDO DANIEL,20137401488,No encontrado,Ciudad Autonoma Buenos Aires
6,SOLIS,EDUARDO DANIEL,20214128323,Santa Fe,Esperanza
7,SOLIS,EDUARDO DANIEL,20236600603,Mendoza,Tunuyan
8,SOLIS,EDUARDO DANIEL,20309576331,No encontrado,No encontrado
9,SOLIS,EDUARDO DANIEL,20329368212,Buenos Aires,Campana
